In [1]:
from pyvi import ViTokenizer
import os
import chromadb
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from llm.llm import BedRockLLMs,CoreLLMs
from llm.llm_utils import *
import boto3    
from dotenv import load_dotenv
load_dotenv()

2024-06-10 17:23:30.632440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 17:23:30.632464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 17:23:30.633046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 17:23:31.180089: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


True

In [2]:
llm = CoreLLMs()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
model = 'bkai-foundation-models/vietnamese-bi-encoder'
embedding = SentenceTransformerEmbeddings(model_name=model)

In [3]:
data_dir="data_raw10k"
persist_dir="/content/drive/MyDrive/speech/RAG/ChromaDB/chromadb"
col_name="wiki10k"
DATA_DIR = data_dir
CHROMA_PATH = persist_dir
COLLECTION_NAME = col_name

vectorstore1 = Chroma(collection_name=COLLECTION_NAME, persist_directory=CHROMA_PATH, embedding_function=embd)

data_dir="gendata"
persist_dir="/content/drive/MyDrive/speech/RAG/ChromaDB/genchromadb"
col_name="wiki10k"
DATA_DIR = data_dir
CHROMA_PATH = persist_dir
COLLECTION_NAME = col_name

vectorstore2 = Chroma(collection_name=COLLECTION_NAME, persist_directory=CHROMA_PATH, embedding_function=embd)

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)

In [1]:
from tqdm import tqdm
texts = []
chunks_ids = []
ids = []
id = 0


for i, file in tqdm(enumerate(os.listdir('data_raw10k'))):
    with open(f'data_raw10k/{file}', 'r') as f:
        data = f.read()
        data = ViTokenizer.tokenize(data)
        
        data = data.split('\n')
        title = data[0]
        data = '\n'.join(data[1:])
        
        docs = text_splitter.split_text(data)
        for doc in docs:
            texts.append(doc)
            ids.append('id'+str(id))
            id += 1
            chunks_ids.append({'chunk':i, 'file':file, 'title':title})
            
set_texts = set()
distinct_texts = []
distinct_ids = []
distinct_chunks_ids = []

for i in range(len(texts)):
    if texts[i] not in set_texts:
        set_texts.add(texts[i])
        distinct_texts.append(texts[i])
        distinct_ids.append(ids[i])
        distinct_chunks_ids.append(chunks_ids[i])
        
batch_size = 100

for i in range(0, len(distinct_texts), batch_size):
    batch_texts = distinct_texts[i:i+batch_size]
    batch_ids = distinct_ids[i:i+batch_size]
    batch_chunks_ids = distinct_chunks_ids[i:i+batch_size]
    vectorstore1.add_texts(batch_texts, ids=batch_ids, metadatas=batch_chunks_ids)
            


In [13]:
for i, file in enumerate(os.listdir('gendata')):
    with open(f'gendata/{file}', 'r') as f:
        data = f.read()
        data = ViTokenizer.tokenize(data)
        
        docs = text_splitter.split_text(data)
        for doc in docs:
            texts.append(doc)
            ids.append('id'+str(id))
            id += 1
            chunks_ids.append({'chunk':i, 'file':file})
            
batch_size = 100

for i in range(0, len(distinct_texts), batch_size):
    batch_texts = distinct_texts[i:i+batch_size]
    batch_ids = distinct_ids[i:i+batch_size]
    batch_chunks_ids = distinct_chunks_ids[i:i+batch_size]
    vectorstore2.add_texts(batch_texts, ids=batch_ids, metadatas=batch_chunks_ids)